---
# AWS S3에 음성파일 업로드 하기
- 리전 설정
- 롱폼 엔진 이용
- pcm -> wav 변환
- 로컬 저장
- 로컬에 저장된 파일을 s3에 업로드
---

In [ ]:
import boto3
import os
import wave
import contextlib

# AWS 액세스 키 및 비밀 액세스 키
AWS_ACCESS_KEY_ID = ''
AWS_SECRET_ACCESS_KEY = ''

# 사용할 Polly 및 S3 리전 설정
polly_region = 'us-east-1'  # Polly 리전
s3_region = 'ap-northeast-2'      # S3 리전

# Polly 및 S3 클라이언트 생성
polly = boto3.client('polly', region_name=polly_region,
                     aws_access_key_id=AWS_ACCESS_KEY_ID,
                     aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
s3 = boto3.client('s3', region_name=s3_region,
                  aws_access_key_id=AWS_ACCESS_KEY_ID,
                  aws_secret_access_key=AWS_SECRET_ACCESS_KEY)

# PCM 파일로 음성을 변환한 후 응답에서 오디오 스트림을 가져옴
def create_and_upload_polly_wav(long_text, file_name, bucket_name):
    try:
        # Polly로 롱폼 텍스트를 음성으로 변환하여 PCM 파일로 저장
        response = polly.synthesize_speech(
            OutputFormat='pcm', # PCM 형식으로 출력
            Text=long_text, # 변환할 텍스트
            VoiceId='Danielle', # 사용할 음성
            Engine='long-form',     # 롱폼 엔진 사용
            TextType='ssml' # 텍스트 형식 (SSML 사용)
        )
        
        # PCM 파일을 WAV 파일로 변환
        pcm_file = f"{file_name}.pcm"  # PCM 파일 이름
        wav_file = f"{file_name}.wav"  # WAV 파일 이름
        with open(pcm_file, 'wb') as f:
            f.write(response['AudioStream'].read()) # 오디오 스트림을 PCM 파일에 씁니다.
        
        # PCM 파일을 WAV 파일로 변환
        with contextlib.closing(wave.open(wav_file, 'w')) as wf:
            wf.setnchannels(1) # 채널 수 설정 (단일 채널)
            wf.setsampwidth(2) # 샘플 폭 설정 (2바이트)
            wf.setframerate(16000) # 샘플링 주파수 설정 (16kHz)
            with open(pcm_file, 'rb') as pcm:
                wf.writeframes(pcm.read()) # PCM 데이터를 WAV 파일에 씁니다.
        
        # WAV 파일을 로컬에 저장할 디렉토리 경로 설정
        local_directory = r""
        
        # WAV 파일을 로컬에 저장
        local_wav_path = os.path.join(local_directory, wav_file)  # 로컬에 저장할 WAV 파일 경로
        os.rename(wav_file, local_wav_path)  # WAV 파일을 지정된 경로로 이동
        print("WAV 파일 로컬에 저장 !!! :", local_wav_path)
        
        # WAV 파일을 S3에 업로드
        s3.upload_file(local_wav_path, bucket_name, wav_file)  # S3에 WAV 파일을 업로드합니다.
        
        # 업로드된 파일의 URL 생성
        file_url = f"https://{bucket_name}.s3.amazonaws.com/{wav_file}"
        print("음성 파일 업로드 !!!")
        print("음성 파일 경로:", file_url)
        
        
    except Exception as e:
        print("에러 발생:", str(e))

# 롱폼 텍스트
long_text = """
<speak>
Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna 
aliqua. Nisl tincidunt eget nullam non. Quis hendrerit dolor magna eget est lorem ipsum dolor sit. Volutpat odio 
facilisis mauris sit amet massa. Commodo odio aenean sed adipiscing diam donec adipiscing tristique.
</speak>
"""

# 파일 이름
file_name = "long_form_audio4"

# S3 버킷 이름
bucket_name = ""

# Polly를 사용하여 롱폼 텍스트를 음성으로 변환하고, 그 결과를 S3에 업로드
create_and_upload_polly_wav(long_text, file_name, bucket_name)